<a href="https://colab.research.google.com/github/dtabuena/Patch_Ephys/blob/main/misc_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def protocol_baseline_and_stim(abf):
    'Return two boolean arrays, distiguishing holding I/V and electrical stimuli'
    # use command signal variance to determine stimulus periods
    commands = []
    for s in abf.sweepList:
        abf.setSweep(sweepNumber=s)
        commands.append(abf.sweepC)
    commands = np.stack(commands)

    std = np.std(commands, axis=0)
    is_base = std==0
    is_stim = np.logical_not(is_base)
    return is_base, is_stim

In [ ]:
def movmean(x, w):
    'A moving mean filter'
    w = int(w)
    # plt.plot(np.arange(len(x)) - int(len(x)/2), x)
    if len(x) < w: w = len(x)
    px = np.pad(x,int(np.ceil((w-1)/2)),'edge')
    if len(px)-len(x) == w: px = px[0:-1]
    conv = np.convolve(px, np.ones(w), 'valid') / w
    return conv